In [1]:
pip install docxtpl

Note: you may need to restart the kernel to use updated packages.


In [2]:
#imports

import os               # File and folder handling
import re               # Regular expressions (used for safe filenames)
import pandas as pd     # DataFrames (Excel/SQL handling)
from docxtpl import DocxTemplate  # For templated Word documents
from datetime import datetime     # For timestamps in letters
from sqlalchemy import create_engine  # To connect Pandas with SQLite DB
import logging   

In [3]:
# CONFIGURATION
# Dictionary of required input files

FILES = {
    'price_list': 'price_list_data.xlsx',
    'promotion': 'Promotion_table_data.xlsx',
    'product_line': 'Product_line_Table.xlsx',
    'sku_table': 'SKU_TABLE.xlsx',
    'seller_mapping': 'Seller Mapping Table.xlsx',
    'category_mapping': 'CATEGORY MAPPING TABLE.xlsx',
    'price_monitoring': 'Price_Monitoring_Table.xlsx',
    'warning_letter_template': 'Warning Letter.docx'
}

In [4]:
# Folder for generated output
OUTPUT_DIR = "output"

In [5]:
# SQLite database name
DB_NAME = "map_compliance.db"

In [6]:
# Ensure output folder exists (no error if it already does)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [7]:
# Create a database connection (SQLite in this case)
engine = create_engine(f"sqlite:///{DB_NAME}", echo=False)

In [8]:
# Configure logging (INFO level = standard runtime messages)
logging.basicConfig(
    format="%(levelname)s: %(message)s",
    level=logging.INFO
)

In [9]:
# HELPER FUNCTIONS

def safe_filename(s):
    """
    Convert any string into a safe filename.
    Removes or replaces characters that would break file saving.
    Example: 'ACME/Store' -> 'ACME_Store'
    """
    return re.sub(r"[^\w\-_. ]", "_", str(s))

In [10]:
# STEP 1: LOAD DATA INTO DB


def load_excel_to_db(name, filepath):
    """
    Load a single Excel file into the SQLite database.
    - name: name of SQL table
    - filepath: path to Excel file
    """
    try:
        # Read the first sheet of the Excel file
        df = pd.read_excel(filepath, sheet_name=0)

        # Strip spaces from column headers
        df.columns = df.columns.str.strip()

        # Save into database as table 'name'
        df.to_sql(name, engine, if_exists="replace", index=False)

        logging.info(f"Loaded {filepath} -> {name}")
    except Exception as e:
        logging.error(f"Failed to load {filepath}: {e}")


def load_all_data():
    """Loop over all Excel files and load them into the DB."""
    for key, path in FILES.items():
        if path.endswith(".xlsx"):   # Only process Excel files
            load_excel_to_db(key, path)



In [11]:
# STEP 2: FIND PRICE VIOLATIONS

def get_violations():
    """
    Query database for violations.
    A violation = Advertised_price < LPP (with a valid date).
    """
    query = """
    SELECT 
        SKU, PL, Category, Sub_category, seller_name, homologated_name,
        MAP_Price, LPP, Advertised_price, Violation_date
    FROM price_monitoring
    WHERE Advertised_price < LPP
      AND Violation_date IS NOT NULL
    """
    return pd.read_sql(query, engine)


In [12]:
# STEP 3: MAP SELLER NAMES


def map_resellers(df):
    """
    Normalize seller names using seller_mapping table.
    If no mapping is found, keep existing homologated_name.
    """
    try:
        mapping = pd.read_sql("SELECT * FROM seller_mapping", engine)
        mapping_dict = dict(zip(mapping['SELLER_NAME'], mapping['HOMOLOGATED_NAME']))
        df['homologated_name'] = df['seller_name'].map(mapping_dict).fillna(df['homologated_name'])
        return df
    except Exception as e:
        logging.warning(f"Seller mapping skipped: {e}")
        return df

In [13]:
# STEP 4: GENERATE WARNING LETTERS

def generate_warning_letters(df):
    """
    Generate Word warning letters for each violation.
    Uses DocxTemplate and 'Warning Letter.docx' template.
    """
    template_path = FILES['warning_letter_template']
    if not os.path.exists(template_path):
        logging.error("Warning letter template not found.")
        return

    template = DocxTemplate(template_path)

    for _, row in df.iterrows():
        # Context dictionary -> keys must match placeholders in template
        context = {
            'Date': datetime.today().strftime('%Y-%m-%d'),
            'Reseller_Name': row['seller_name'],
            'Reseller_Company': row['homologated_name'],
            'Product_Name': f"{row['SKU']} ({row['Sub_category']})",
            'MAP_Price': f"${row['MAP_Price']:,.2f}",
            'Advertised_Price': f"${row['Advertised_price']:,.2f}",
            'Date_of_Violation': row['Violation_date'],
            'Platform': row['seller_name'],
            'Contact_Information': 'legal@client.com | +1-800-MAP-POLICY'
        }

        # Fill template with context
        template.render(context)

        # Safe file name: WARNING_{homologated_name}_{SKU}.docx
        filename = os.path.join(
            OUTPUT_DIR,
            f"WARNING_{safe_filename(row['homologated_name'])}_{row['SKU']}.docx"
        )

        # Save filled document
        template.save(filename)
        logging.info(f"Generated {filename}")

In [14]:
# STEP 5: FLAG REPEAT VIOLATORS

def flag_suspensions():
    """
    Detect resellers with >1 violation per category.
    Save them into 'suspension_flagged' table for audit.
    """
    query = """
    SELECT homologated_name, Category, COUNT(*) as violation_count
    FROM price_monitoring
    WHERE Advertised_price < LPP
      AND Violation_date IS NOT NULL
    GROUP BY homologated_name, Category
    HAVING COUNT(*) > 1
    """
    susp = pd.read_sql(query, engine)

    # Save results into DB
    susp.to_sql("suspension_flagged", engine, if_exists="replace", index=False)

    if susp.empty:
        logging.info("No repeat violations found.")
    else:
        for _, row in susp.iterrows():
            logging.warning(
                f"Suspend {row['homologated_name']} in {row['Category']} "
                f"(Violations: {row['violation_count']})"
            )


In [15]:
# MAIN PROGRAM

def main():
    logging.info("Starting MAP Compliance System...")

    # Step 1: Load Excel data into DB
    load_all_data()

    # Step 2: Find violations
    viol = get_violations()
    if viol.empty:
        logging.info("No violations found. Process complete.")
        return

    # Step 3: Map resellers
    viol = map_resellers(viol)

    # Step 4: Generate warning letters
    generate_warning_letters(viol)

    # Step 5: Flag suspensions
    flag_suspensions()

    logging.info("Process complete.")


# Run program if script executed directly
if __name__ == "__main__":
    main()

INFO: Starting MAP Compliance System...
INFO: Loaded price_list_data.xlsx -> price_list
INFO: Loaded Promotion_table_data.xlsx -> promotion
INFO: Loaded Product_line_Table.xlsx -> product_line
INFO: Loaded SKU_TABLE.xlsx -> sku_table
INFO: Loaded Seller Mapping Table.xlsx -> seller_mapping
INFO: Loaded CATEGORY MAPPING TABLE.xlsx -> category_mapping
INFO: Loaded Price_Monitoring_Table.xlsx -> price_monitoring
INFO: Generated output\WARNING_Publix Super Markets_4K0V8PA.docx
INFO: Generated output\WARNING_Casey_s General Store_03V96QB.docx
INFO: Generated output\WARNING_My Demoulas_952X8QB.docx
INFO: Generated output\WARNING_Lululemon_6X4B9PA.docx
INFO: Generated output\WARNING_Walgreens Boots Alliance_3ED50A.docx
INFO: Generated output\WARNING_Wegmans Food Market_W2090A.docx
INFO: Generated output\WARNING_Lowe_s Companies_W1370A.docx
INFO: Generated output\WARNING_Good Neighbor Pharmacy_492W7B.docx
INFO: Generated output\WARNING_Signet Jewelers_P2V64A.docx
INFO: Generated output\WARNING